In [4]:
import nltk
from nltk.wsd import lesk

import support.scrape as scrape

from pywsd.similarity import max_similarity,similarity_by_path
from pywsd.lesk import simple_lesk, adapted_lesk, cosine_lesk,original_lesk

from nltk.tokenize import PunktSentenceTokenizer,sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet as wn
import string
import operator


from frenetic.frenetic import *

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

# Languages
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer


#targetLang = "english"
targetLang = "french"


fwn = None
if targetLang == "french":
    fwn = FreNetic("./frenetic/data/wolf.xml") 


stop_words = set(stopwords.words(targetLang)) 

def simpleFilter(sentence):
    filtered_sent = []
    
    lemmatizer = None
    if targetLang == "french":
        lemmatizer = FrenchLefffLemmatizer()
    else:
        lemmatizer = WordNetLemmatizer()
    
    
    stop_words = set(stopwords.words(targetLang))
    words = word_tokenize(sentence)

    for w in words:
        if w not in stop_words:
            filtered_sent.append(lemmatizer.lemmatize(w))

    return filtered_sent

def listToString(inputSentList):
    return ' '.join(inputSentList)

def viewAllDefinitions(wordStr):
    resList = []
    for syn in wn.synsets(wordStr):
        resList.append(syn.definition())
    return resList

def topRelevSent(targetWord):
    return

def simpleFilter(sentence):
    filtered_sent = []
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words(targetLang))
    words = word_tokenize(sentence)

    for w in words:
        if w not in stop_words:
            filtered_sent.append(lemmatizer.lemmatize(w))
    return listToString(filtered_sent)

def simpleFilterList(sentList):
    resList = []
    for sent in sentList:
        sent = sent.translate(str.maketrans('', '', string.punctuation))
        resList.append(simpleFilter(sent))
    return resList

def combineWordVec(a,b,aVec,bVec):
    aDf = pd.DataFrame(a.toarray(),columns=aVec.get_feature_names())
    bDf = pd.DataFrame(b.toarray(),columns=bVec.get_feature_names())

    combinedDf = pd.concat([aDf, bDf],sort=False).fillna(value=0.0)
    aRes = combinedDf.iloc[0].values
    bRes = combinedDf.iloc[1].values
    return aRes,bRes
    
def calcCosSim(a,b):
    # Calculate cosine simularity
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    
def leskAlgoFr(sentence,ambiguous):
    synsets = fwn.synsets("cœur")
    
    sentence = simpleFilter(sentence)
    
    resAr = []
    for test in synsets:
        # We calculate cosine simularity
        vecSent = CountVectorizer()
        sentenceFreq = vecSent.fit_transform([sentence])
        
        definition = test.defn()
        
        vecDef = CountVectorizer()
        definitionFreq = vecDef.fit_transform([definition])
        
        defRes,sentRes = combineWordVec(definitionFreq,sentenceFreq,vecDef,vecSent)
        
        res = calcCosSim(defRes,sentRes)
        resAr.append(res)
        
    return synsets[np.argmax(np.asarray(resAr))].defn()



'''
sent = 'His card was the 8 of hearts'
ambiguous = 'heart'
'''


sent = 'Sa carte était la huit de cœur'
ambiguous = 'cœur'


sent = simpleFilter(sent)


#cosine_lesk(sent,ambiguous).definition()
print(sent)
print(cosine_lesk(sent,ambiguous))


# original_lesk seems to be better

Warming up PyWSD (takes ~10 secs)... took 11.802986145019531 secs.


Sa carte huit cœur
None


In [5]:

leskAlgoFr(sent,ambiguous)

'une carte à jouer dans le costume majeur qui a un ou plusieurs coeurs rouges dessus'

In [6]:
synsets = fwn.synsets("cœur")

for test in synsets:
    print(test.sid())
    print(test.defn())

eng-30-03107904-n
la chambre d'un réacteur nucléaire contenant la matière fissile où la réaction a lieu
eng-30-03507048-n
une carte à jouer dans le costume majeur qui a un ou plusieurs coeurs rouges dessus
eng-30-04624826-n
une inclination ou tendance d'un certain genre
eng-30-04857490-n
le courage de continuer
eng-30-05388805-n
l'organe musculaire creux situé derrière le sternum et entre les poumons; ses contractions rythmiques déplacent le sang à travers le corps
eng-30-05919263-n
the locus of feelings and intuitions
eng-30-05921123-n
the choicest or most essential or most vital part of some idea or experience
eng-30-07544647-n
a positive feeling of liking
eng-30-07651905-n
a firm rather dry variety meat (usually beef or veal)
eng-30-13865904-n
a plane figure with rounded sides curving inward at the top and intersecting at the bottom; conventionally used on playing cards and valentines


In [7]:

        
viewAllDefinitions('bank')     


['sloping land (especially the slope beside a body of water)',
 'a financial institution that accepts deposits and channels the money into lending activities',
 'a long ridge or pile',
 'an arrangement of similar objects in a row or in tiers',
 'a supply or stock held in reserve for future use (especially in emergencies)',
 'the funds held by a gambling house or the dealer in some gambling games',
 'a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force',
 'a container (usually with a slot in the top) for keeping money at home',
 'a building in which the business of banking transacted',
 'a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)',
 'tip laterally',
 'enclose with a bank',
 'do business with a bank or keep an account at a bank',
 'act as the banker in a game or in gambling',
 'be in the banking business',
 'put into a bank account',
 'cover with ashes so to co

In [ ]:
# nltk
sent = 'I went to the bank to deposit my money'
ambiguous = 'bank'
original_lesk(sent,ambiguous).definition()

In [ ]:
original_lesk('my cat likes to eat mice', 'cat').definition()

In [ ]:
#sent = 'She is mean to me'
#sent = 'She mean me'
sent = 'What do you mean'

original_lesk(sent, 'mean').definition()
#simple_lesk(sent, 'mean').definition()

In [ ]:
# 'run'  'advice' , 'bat', 'project'  'fine'  'consoil'  'roll'  ''
sent = 'The rebel seized the opportunity to rebel'
adapted_lesk(sent, 'rebel').definition()

In [13]:
def seperateByDef(targetWord):
    # Returns a dictionary sorted by defintion
    
    sentList = None
    if targetLang == "french":
        
        sentList = scrape.scrapeFR(targetWord)
    else:
        sentList = scrape.scrape(targetWord)
    
    dictDef = {}
    for i,sent in enumerate(sentList):
        defineSent = None
        if targetLang == "french":
            defineSent = leskAlgoFr(sent,ambiguous) 
        else:
            #defineSent = cosine_lesk(sent,targetWord).definition()
            defineSent = adapted_lesk(sent,targetWord).definition()        
        
        if defineSent not in dictDef:
            dictDef[defineSent] = [sent]
        else:
            dictDef[defineSent].append(sent)
    return dictDef

def printDictKeysVals(inputDict):
    for key in inputDict.keys():
        print('-------------------------------------------------------')
        print('Def: ', key)
        for value in inputDict[key]:
            print('        ', value)
    
def getDef(sent,targetWord):
    # Get defintion of word
    
    if targetLang == "french":
        defineSent = leskAlgoFr(sent,ambiguous) 
    else:
        #defineSent = cosine_lesk(sent,targetWord).definition()
        #defineSent = adapted_lesk(sent,targetWord).definition() 
        defineSent = original_lesk(sent,targetWord).definition() 
    
    return defineSent

def printDefExampleSent(inputDict,wordDef):
    print('Definition: ')
    print(wordDef)
    print('\n\n\n')
    print('Example sentence:')
    
    
    
    if wordDef not in inputDict:
        print('No example sentences found')
        return
    
    for sent in inputDict[wordDef]:
        print('-',sent)
        print('')
        
        
def countDefFreq(inputDict,n):
    resDict = {}
    for key in inputDict.keys():
        resDict[key] = len(inputDict[key])
    return getTopDefEntries(resDict, n)

def getTopDefEntries(inputDict, n):
    return dict(sorted(inputDict.items(), key=operator.itemgetter(1), reverse=True)[:n])

 
targetWord = 'cœur'
#testSent = 'He does not have money in his bank account'
testSent = 'Sa carte était la huit de cœur'



dictDef = seperateByDef(targetWord)
wordDef = getDef(testSent,targetWord)

#printDefExampleSent(dictDef,wordDef)
#printDictKeysVals(dictDef)



# Print out frequency of each definiton used
topRes = countDefFreq(dictDef, 5)


In [14]:
for key in topRes.keys():
    print(key,': ', topRes[key])

l'organe musculaire creux situé derrière le sternum et entre les poumons; ses contractions rythmiques déplacent le sang à travers le corps :  12
la chambre d'un réacteur nucléaire contenant la matière fissile où la réaction a lieu :  11
une inclination ou tendance d'un certain genre :  2
le courage de continuer :  1
une carte à jouer dans le costume majeur qui a un ou plusieurs coeurs rouges dessus :  1


In [15]:
dictDef[list(topRes.keys())[0]]

['Les chemins de fer ont été au cœur de la révolution industrielle.',
 'En ce qui concerne, notamment, la qualité du matériau à utiliser, les deux qualités (cœur blanc et cœur noir) sont autorisées.',
 "Chacune d'entre elles doit se concentrer sur ses qualités et son savoir-faire propres - sur son cœur de métier.",
 "Le cœur de l'enseignement de Jésus, avec ses demandes exigeantes en ce qui regarde le Royaume de Dieu, est un appel à la metanoia, même si le terme « conversion » n'est pas toujours employé.",
 "Le développement de l'Espace Européen de la Recherche et de l'Innovation nous tient à cœur afin de rendre l'Europe plus compétitive, une Europe basée sur la connaissance et l'innovation.",
 "Le corps matériel se recroquevilla suite à ce retour précipité de l'esprit, si bien qu'il se mit à suer et que son cœur commença à battre la chamade.",
 "Il est temps de prendre les choses à cœur et de montrer la volonté politique nécessaire qui nous permettra de consolider la base de l'élimina

In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation
from nltk.stem.snowball import SnowballStemmer

punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = SnowballStemmer("english")

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

def get_clusters(sentences):
    tf_idf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = (tf_idf_matrix * tf_idf_matrix.T).A
    affinity_propagation = AffinityPropagation(damping=0.5)
    affinity_propagation.fit(similarity_matrix)
    print(similarity_matrix.shape)

    labels = affinity_propagation.labels_
    cluster_centers = affinity_propagation.cluster_centers_indices_

    tagged_sentences = zip(sentences, labels)
    clusters = {}

    for sentence, cluster_id in tagged_sentences:
        clusters.setdefault(sentences[cluster_centers[cluster_id]], []).append(sentence)

    return clusters, affinity_propagation

sentences = scrape.scrape('cat')


#sentences = simpleFilterList(sentences)

clusters, affinity_propagation = get_clusters(sentences)
print(len(clusters))
for cluster in clusters:
    print('--------------------------------------')
    print(cluster, ':')
    for element in clusters[cluster]:
        print('  - ', element)

tf_idf_predMat = vectorizer.fit_transform(['The cat is running away from the dog'])
similarity_matrixPred = (tf_idf_predMat.T*tf_idf_predMat).A
#print(affinity_propagation.predict(similarity_matrixPred))
print(similarity_matrixPred.shape)

In [ ]:
vectorizer0 = TfidfVectorizer(tokenizer=normalize)
sentences = scrape.scrape('cat')

tf_idf_matrix0 = vectorizer0.fit_transform(sentences)
similarity_matrixCluster = (tf_idf_matrix0.T * tf_idf_matrix0).A
affinity_propagation0 = AffinityPropagation(damping=0.5)
affinity_propagation0.fit(similarity_matrixCluster)
        

    
vectorizer = TfidfVectorizer(tokenizer=normalize)
tf_idf_matrix = vectorizer.fit_transform(['The cat is running away from the dog'])
similarity_matrixPred = (tf_idf_predMat.T*tf_idf_predMat).A
#print(affinity_propagation.predict(similarity_matrix))



In [ ]:
import pandas as pd
import numpy as np
similarity_matrixPredFlat = np.sum(similarity_matrixPred,axis=0)/similarity_matrixPred.shape[0]


predDf = pd.DataFrame(columns=vectorizer.get_feature_names())
predDf.loc[len(predDf)] = similarity_matrixPredFlat

similarity_matrixCluster = np.sum(similarity_matrixCluster,axis=0)/similarity_matrixCluster.shape[0]

clusterDf = pd.DataFrame(columns=vectorizer0.get_feature_names())
clusterDf.loc[len(clusterDf)] = similarity_matrixCluster

combinedDf = pd.concat([predDf , clusterDf],sort=False).fillna(value=0.0)

predVec = combinedDf.iloc[0].values
clusterVec = combinedDf.iloc[1].values

In [ ]:
print(similarity_matrixCluster.shape)
#print(affinity_propagation.predict(predVec))


In [ ]:
tfPredMat = predVec.tolist()*predVec.transpose()
predVec.shape